# MA5701 Optimización no lineal: Tarea 3
**Fecha de entrega:** 4 de Julio a las 23:59 hrs.

**Profesor:** Jorge G. Amaya A. 

**Auxiliar:** Aldo Gutiérrez Concha. 

**Ayudantes:** Carolina Chiu y Mariano Vazquez.

**Autor:** Felipe Urrutia Vargas

In [2]:
!pip install numdifftools

In [3]:
import numpy as np
import numdifftools as nd

In [4]:
def f1(x):
    return 8 * ((x[0] - 6)**2) + (x[1] - 2)**4

In [7]:
grad = nd.Gradient(f1)
print(grad([0, 2]))

[-96.   0.]


In [52]:
A = np.array([
    [-1, 2],
    [3, 2],
    [-1, 0],
    [0, -1]
])
b = np.array([4, 12, 0, 0])

E = np.array([])
e = np.array([])

## I. Escriba un código computacional que haga operacional el método de direcciones admisibles (Zoutendijk)

Resolver el problema de optimización no lineal:
$$(P) \quad \begin{cases} \min &f (x) \\
s.a. &Ax \leq b \\
&Ex = e
\end{cases}$$

### Pseudo-algoritmo

(0) Sean $\epsilon>0$, $k = 0$, $x_0 \in \mathbb{R}^n$ tal que $Ax_0 \leq  b$, $Ex_0=e$ 

In [53]:
eps= 1e-2
x0 = [0, 2]
k = 0
xk = x0
# check factibilidad
feasible = np.all(A@xk <= b)
print(f"(0, k={k}) Factible: {feasible}")

(0, k=0) Factible: True


(1) Sea la descomposición $A = [A_1,A_2]^T, b =(b_1, b_2)^T$ tal que $A_1x_k = b_1, A_2x_k < b_2$.

In [54]:
# Step 1
index_A1 = np.isclose(A@xk, b)
index_A2 = ~index_A1

A1 = A[index_A1]
A2 = A[index_A2]

b1 = b[index_A1]
b2 = b[index_A2]

print(f"A1:\n{A1},\nb1:{b1}")
print(f"A2:\n{A2},\nb1:{b2}")

A1:
[[-1  2]
 [-1  0]],
b1:[4 0]
A2:
[[ 3  2]
 [ 0 -1]],
b1:[12  0]


(2) Resolver el problema lineal

$$(D_k) \quad \begin{cases} \min &\nabla f (x_k)^T d\\
s.a. &A_1d \leq 0 \\
&Ed = 0\\
&−1 \leq d_j \leq 1, \quad j = 1, . . . , n
\end{cases}$$
y sea $d_k$ solución de $(D_k)$.

Si $|| \nabla f (x_k)^T d_k || < \epsilon$, entonces parar.

Si no, ir a (3).

In [43]:
from scipy.optimize import linprog

In [74]:
# Step 2

obj = grad([0, 2])
print(f"∇𝑓(𝑥𝑘): {obj}")

lhs_ineq = A1 if len(A1) != 0 else None

rhs_ineq = np.zeros(A1.shape[0]) if len(A1) != 0 else None

lhs_eq = E if len(E) != 0 else None
rhs_eq = np.zeros(E.shape[0]) if len(E) != 0 else None

bnd = [(-1, 1) for _ in range(obj.shape[0])] 

opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
              A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd,
              method="simplex")
dk = opt.x
print(f"dk: {dk}")

∇𝑓(𝑥𝑘): [-96.   0.]
dk: [1.  0.5]


(3) Determinar el paso, resolviendo aproximadamente el problema de minimización unidimensional

$$(L) \quad \begin{cases} \min &f (x_k + \lambda d_k)\\
s.a. &\lambda \in [0, \bar{\lambda_k}]
\end{cases}$$

mediante el método de Armijo.

Se usa 

$$ \bar{\lambda_k} = \min \left\lbrace \frac{(b_2 − A_2 x_k)_i}{(A_2 d_k)_i} : (A_2 d_k)_i > 0 \right\rbrace$$
y se considera $\bar{\lambda_k} = +\infty$ cuando $(A_2 d_k)_i \leq 0 \forall i$.

Sea $\lambda_k$ la solución del subproblema $(L)$. Hacer:

$$x_k+1 = x_k + \lambda_k d_k,$$
$$k ← k + 1,$$
$$\text{e ir a $(1)$.}$$

## II. El paso de Armijo debe programarlo usted como una rutina que será llamada en cada iteración del algoritmo de Zoutendijk

In [176]:
# Step 3

dom = A2@dk>0
bar_lmbdk = np.min(((b2-A2@xk)/(A2@dk))[dom]) if len(dom) != 0 else float("inf")
print(f"bar 𝜆𝑘: {bar_lmbdk}")

d_phi_0 = nd.Gradient(lambda t: f1( xk + dk*t ))([0])

N = 10*6
t = np.linspace(0, bar_lmbdk, N)
h = 1/(N-1)
sgm=0.9

dom = np.where(f1( (xk + dk * t[:, None]).T ) <= f1(xk) + sgm*d_phi_0*t)[0]
ix_max = np.max(dom)
lmbdk = ix_max * h * bar_lmbdk
print(f"𝜆𝑘: {lmbdk}")

xk1 = xk + lmbdk * dk
print(f"xk+1: {xk1}") 

bar 𝜆𝑘: 2.0
𝜆𝑘: 1.1864406779661016
xk+1: [1.18644068 2.59322034]
